In [1]:
import pandas as pd
from oracle import oracleTopd

data = oracleTopd('select * from recipe_data_table where rownum <= 1000') # 데이터 불러오기 (1000개, 15초)

ModuleNotFoundError: No module named 'oracle'

In [69]:
import random 
import numpy as np

np.random.seed(77)
toy = data[["recipe_title", "recipe_step", "recipe_ingredients"]] # toy 데이터 만들기 표본 1000개
toy.head()

,recipe_title,recipe_step,recipe_ingredients
0,전복 탕수육 만들기,"['#1 먼저, 전복 8마리를 준비해줍니다. 양이 모자랄까 걱정했는데, 두 명이 밥...","{'재료': ['전복8마리', '튀김가루2큰술', '물6큰술', '오이1/4개', ..."
1,대패삼겹살튀김 독특한 요리,"['#1 당근은 가늘게 채썰어 주세요', '#2 버섯도 얇게 편 썰어주세요', '#...","{'재료': ['대패삼겹살150g', '당근1/2개', '콩송이버섯1줌', '계란1..."
2,팬더 익스프레스의 오렌지 치킨 만들기,"['#1 오렌지 소스의 물, 간장, 설탕, 식초, 미림, 전분을 섞어둔다.', '#...","{'재료': ['닭고기300g', '소금1/4t', '후추약간', '올리브오일1t'..."
3,"추석요리, 탱탱하고 쫄깃한 식감의 새우튀김 & 쥐포튀김",['#1 재료를 준비합니다.쥐포는 물에 5분정도 담가두세요.새우는 해동시켜 맛술 1...,"{'재료': ['쥐포2장', '새우5미', '밀가루1컵', '빵가루1컵', '물1컵..."
4,요알못도 할 수 있는 크리스피 오겹살 레시피,['#1 오겹살을 준비합니다.삼겹과 오겹의 차이는 껍질 유무입니다.우리는 껍질을 튀...,"{'재료': ['돼지고기800g', '소금1소끔씩', '올리브오일5-8스푼']}"


In [8]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import re


# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()

In [70]:
okt = Okt()
tagged_corpus_list = []
cleaned_tagged_corpus_list = []

for index, row in tqdm(toy.iterrows(), total=len(toy)):
    cleaned_text = re.sub(r'[^A-Za-z0-9가-힣\s]+', '', str(row['recipe_step']))
    words = okt.morphs(cleaned_text)
    tag = row['recipe_title']

    # recipe_step의 길이가 0이 아니면 다음 작업을 수행합니다.
    if len(words) != 0:
        tagged_corpus_list.append(TaggedDocument(words=words, tags=[tag]))


100%|██████████| 1000/1000 [00:44<00:00, 22.56it/s]


In [38]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)
model.build_vocab(tagged_corpus_list) # Vocabulary 빌드

print(f"Tag Size: {len(model.dv.key_to_index)}", end=' / ') # 수정된 부분: doctags 대신 key_to_index 사용

model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=10) # Doc2Vec 학습
model.save('recipe.doc2vec') # 모델 저장


Tag Size: 932 / 

In [80]:
tagged_corpus_list[:3]

[TaggedDocument(words=['1', '먼저', '전복', '8', '마리', '를', '준비', '해줍니다', '양', '이', '모자랄까', '걱정', '했는데', '두', '명', '이', '밥', '반찬', '으로', '먹기', '딱', '알맞은', '양', '이', '되더라구요', '2', '전', '복솔', '을', '이용', '해', '전복', '을', '깨끗이', '문질러', '씻어', '줍니다', '칫솔', '을', '이용', '해도', '되는데', '마침', '전', '복솔', '도', '함께', '보내주셔서', '이용', '해', '봤어요', '3', '숟가락', '을', '이용', '해', '전복', '살', '과', '껍데기', '를', '분리', '한', '후', '전복', '앞', '부분', '의', '빨간색', '이빨', '을', '칼', '로', '잘라', '냅니다', '4', '이제', '칼', '을', '이용', '해', '전복', '살', '과', '내장', '을', '분리', '해줍니다', '이', '때', '내장', '이', '터지지', '않도록', '주의', '해서', '분리', '해주셔야', '해', '요', '전복', '살', '은', '탕수육', '에', '쓸거구요', '전복', '내장', '은', '전복죽', '에', '쓸거예요', '5', '손질', '을', '마친', '전복', '살', '은', '칼집', '을', '내어', '주었구요', '키친', '타', '올로', '물기', '를', '꼼꼼히', '닦아주었어요', '또', '탕수육', '소스', '에', '쓸', '오이', '당근', '양파', '를', '썰어', '주었어요', '6', '튀김', '가루', '물', '을', '넣고', '잘', '섞어', '튀김', '반죽', '을', '만들어', '줍니다', '여기', '에', '손질', '한', '전복', '을', '넣어', '줍니다', '7', '후라이팬', '에', '포도', '씨유',

In [88]:
list(model.dv.key_to_index.keys)

TypeError: 'builtin_function_or_method' object is not iterable

In [90]:
similar_doc = model.dv.most_similar('전복 탕수육 만들기')
print(similar_doc)

[('(집에서 즐기는 술안주) 닭똥집튀김', 0.7849906086921692), ('오징어탕수육', 0.7761306166648865), ('야식이나 맥주안주로 스팸튀김 꼬치', 0.7730869054794312), ('집에서 닭튀김 :) 깐풍소스와 함께 먹어요 !', 0.7661173343658447), ('씹고 뜯고 맛보고! 역시 치킨은 만화치킨이지! 만화속닭다리치킨', 0.761171281337738), ('굴전', 0.7398727536201477), ('매콤한 핫 깐풍기', 0.736855685710907), ('두반장소스 활용 요리/ 매운 조기탕수', 0.731732964515686), ('이길수 밖에 없는 조합 "마약 닭 다리" 레시피', 0.7288693785667419), ('고구마롤 돈까스', 0.7259566187858582)]


In [92]:
similar_doc_2 = model.dv.most_similar('대패삼겹살튀김 독특한 요리')
print(similar_doc_2)

[('가지튀김/백종원레시피', 0.8012381792068481), ('고구마롤 돈까스', 0.7966592907905579), ('해피투게더 야간매점 이민우 마마밥', 0.7921081781387329), ('든든한 영양간식~ 달걀돈까스', 0.7920870780944824), ('진짜 오징어 튀김을 상큼한 소스와 함께! 크리미어니언오징어링', 0.7861824035644531), ('탕수육', 0.7842554450035095), ('닭가슴살 치킨까스', 0.7822151780128479), ('닭가슴살 치킨까스 만들기', 0.7818322777748108), ('술안주로 딱 좋은 스위트콘전 레시피!', 0.7778521776199341), ('겉바속촉 애호박새우튀김!', 0.7778517007827759)]


### 해결해야할 과제


1. 레시피에서 하는 요리에 대한 정의가 필요함
  - 레시피 타이틀 전처리
  - 레시피 태그 전처리 
2. 레시피에 따라서 2인분, 3인분, 4인분 다양함

In [65]:
data["recipe_summary1"].value_counts()

2인분       57219
4인분       31408
1인분       25937
3인분       19687
6인분 이상    16377
5인분        3670
Name: recipe_summary1, dtype: int64

### 영양소를 고려한 임베딩

In [177]:
toy = data[["recipe_title", "recipe_ingredients"]].iloc[:1000].copy()

In [178]:
# 식재료 칼럼 만들기 (20개)
for i in range(1, 21):
    toy.loc[:, f'ingredient{i}'] = None
    toy.loc[:, f'quantity{i}'] = None
    toy.loc[:, f'unit{i}'] = None


In [191]:
import tqdm
import re
import ast
non_matching_items =[]
for idx, row in tqdm.tqdm(toy.iterrows(), total=toy.shape[0]):
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])
    ingredient_count = 1
    for category, items in ingredients_dict.items():
        if items:  # 아이템이 존재하는 경우
            for item in items:
                # 숫자가 포함된 경우와 숫자가 없는 경우 모두 처리
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    toy.at[idx, f'ingredient{ingredient_count}'] = ingredient
                    toy.at[idx, f'quantity{ingredient_count}'] = quantity if quantity else ""  # 숫자가 없는 경우 빈 문자열 할당
                    toy.at[idx, f'unit{ingredient_count}'] = unit if unit else ""  # 단위가 없는 경우 빈 문자열 할당

                    ingredient_count += 1
                else:
                    non_matching_items[idx] = item

for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')



100%|██████████| 1000/1000 [00:03<00:00, 254.42it/s]

Row 34: ​고추가루1/2큰술
Row 258: (소주or맛술)1큰술
Row 302: ﻿대파조금
Row 367: 18배멸치육수2T
Row 424: ​간장2수저
Row 598: 
Row 837: 
Row 892: 
Row 895: ​식용유1큰술
